In [6]:
# First time instalation needed
pip install ipython-sql

In [4]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

In [5]:
%%sql
SELECT *
  FROM sqlite_master
 WHERE type='table';

 * sqlite:///factbook.db
Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


In [6]:
%%sql
SELECT *
  FROM facts
LIMIT 5;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


name — the name of the country.

area— the country's total area (both land and water).

area_land — the country's land area in square kilometers.

area_water — the country's waterarea in square kilometers.

population — the country's population.

population_growth— the country's population growth as a percentage.

birth_rate — the country's birth rate, or the number of births per year per 1,000 people.

death_rate — the country's death rate, or the number of death per year per 1,000 people.

### Population stats

In [17]:
%%sql
SELECT 
    MIN(population) as min_population,
    MAX(population) as max_population,
    MIN(population_growth) as min_population_growth,
    MAX(population_growth) as max_population_growth
FROM facts;

 * sqlite:///factbook.db
Done.


min_population,max_population,min_population_growth,max_population_growth
0,7256490011,0.0,4.02


#### Exploring Countries Maximum and minimum population

In [9]:
%%sql
SELECT 
    id, 
    code, 
    name
    from facts
where population == (SELECT MIN(population) from facts);

 * sqlite:///factbook.db
Done.


id,code,name
250,ay,Antarctica


In [11]:
%%sql
SELECT 
    id, 
    code, 
    name
    from facts
where population == (SELECT MAX(population) from facts);

 * sqlite:///factbook.db
Done.


id,code,name
261,xx,World


The table contains an entry for Antarctica and another for all the world population, to observe and analyze better the stats, the previous entries will be discarded.

In [19]:
%%sql
SELECT 
    MIN(population) as min_population,
    MAX(population) as max_population,
    MIN(population_growth) as min_population_growth,
    MAX(population_growth) as max_population_growth
    FROM facts
WHERE name != 'Antarctica' 
    AND name != 'World';

 * sqlite:///factbook.db
Done.


min_population,max_population,min_population_growth,max_population_growth
48,1367485388,0.0,4.02


As we can observe the stats now had changed containg values that have sense.

In [24]:
%%sql
SELECT 
    ROUND(AVG(population), 0) as average_population,
    ROUND(AVG(area), 0) as average_area    
    FROM facts
WHERE name != 'Antarctica' 
    AND name != 'World';

 * sqlite:///factbook.db
Done.


average_population,average_area
32377011.0,555094.0


### Explorig Population Density Statistics

In this section, we're going to find out which countries are the most densely populated and which ones – the least.

First, let's calculate the average population density in the world, which practically means dividing population by area. Here and later, we have to:

<li>Use area_land instead of area, for being more precise.</li>
<li>exclude null values for both population and area_land.</li>
<li>exclude area_land=0 to avoid having 0 in the denominator.</li>
<li>exclude the outliers: World, European Union, and Antarctica.</li>
<li>Let's look once again at the outliers.</li>

In [25]:
%%sql
SELECT name, population, area_land
  FROM facts
WHERE 
    name = 'World' OR 
    name = 'European Union' OR 
    name = 'Antarctica';

 * sqlite:///factbook.db
Done.


name,population,area_land
European Union,513949445,None
Antarctica,0,280000
World,7256490011,None


So, according to the statements above, we have to keep in mind only the row for Antarctica, since the other 2 outliers have null values for area_land.

Now we're ready to calculate the average values for the population density, population and area.

### Countries above average

In [30]:
%%sql
SELECT name, population, area_land, 
       ROUND(CAST(population AS Float)/CAST(area_land AS Float), 0) 
                                          AS population_density
    FROM facts
WHERE name != 'Antarctica'      
  AND population NOT NULL 
    AND area_land NOT NULL
      AND area_land != 0
        AND population > (
            SELECT AVG(population)
              FROM facts
            WHERE name != 'Antarctica'
              AND population NOT NULL 
                AND area_land NOT NULL
                  AND area_land != 0)
        AND area_land < (
            SELECT AVG(area_land)
              FROM facts
            WHERE name != 'Antarctica'
              AND population NOT NULL 
                AND area_land NOT NULL
                  AND area_land != 0)
ORDER BY population_density DESC;

 * sqlite:///factbook.db
Done.


name,population,area_land,population_density
Bangladesh,168957745,130170,1298.0
"Korea, South",49115196,96920,507.0
Japan,126919659,364485,348.0
Philippines,100998376,298170,339.0
Vietnam,94348835,310070,304.0
United Kingdom,64088222,241930,265.0
Germany,80854408,348672,232.0
Nepal,31551305,143351,220.0
Italy,61855120,294140,210.0
Uganda,37101745,197100,188.0


In [ ]:
https://nbviewer.jupyter.org/urls/community.dataquest.io/uploads/short-url/3bAfruaNQBp7mLwVmbUAsQ8dgsx.ipynb

### Country with most population

In [37]:
%%sql
SELECT name, population
    FROM facts
WHERE population == (SELECT MAX(population)
                        FROM facts
                    WHERE name != 'World' 
                        AND name != 'European Union'
                            AND name != 'Antarctica');

 * sqlite:///factbook.db
Done.


name,population
China,1367485388


### Country with highest growth rate

In [38]:
%%sql
SELECT name, population_growth
    FROM facts
WHERE population_growth == (SELECT MAX(population_growth)
                        FROM facts
                    WHERE name != 'World' 
                        AND name != 'European Union'
                            AND name != 'Antarctica');

 * sqlite:///factbook.db
Done.


name,population_growth
South Sudan,4.02


### Which countries will add the most people to their populations next year?

In [46]:
%%sql
SELECT name, birth_rate
  FROM facts
    WHERE birth_rate > (SELECT AVG(birth_rate)
                       FROM facts)
ORDER BY birth_rate DESC
LIMIT 5;

 * sqlite:///factbook.db
Done.


name,birth_rate
Niger,45.45
Mali,44.99
Uganda,43.79
Zambia,42.13
Burkina Faso,42.03


### Countries with higher death rate than birth rate

In [47]:
%%sql
SELECT name, birth_rate
  FROM facts
      WHERE death_rate > birth_rate
ORDER BY birth_rate DESC
LIMIT 5;

 * sqlite:///factbook.db
Done.


name,birth_rate
Moldova,12.0
Russia,11.6
Ukraine,10.72
Belarus,10.7
Estonia,10.51


### Countries with the highest ratios wtaer / land

In [40]:
%%sql
SELECT name, area_water, area_land,
      ROUND(CAST(area_water AS Float)/ CAST(area_land AS Float), 1) AS water_to_land
    FROM facts
WHERE area_land != 0 
  AND area_water NOT NULL
ORDER BY water_to_land DESC
LIMIT 10;

 * sqlite:///factbook.db
Done.


name,area_water,area_land,water_to_land
British Indian Ocean Territory,54340,60,905.7
Virgin Islands,1564,346,4.5
Puerto Rico,4921,8870,0.6
"Bahamas, The",3870,10010,0.4
Guinea-Bissau,8005,28120,0.3
Malawi,24404,94080,0.3
Eritrea,16600,101000,0.2
Liberia,15049,96320,0.2
Netherlands,7650,33893,0.2
Uganda,43938,197100,0.2


### Countries with more water than land

In [42]:
%%sql
SELECT name, area_water, area_land      
    FROM facts
WHERE area_water > area_land
LIMIT 10;

 * sqlite:///factbook.db
Done.


name,area_water,area_land
British Indian Ocean Territory,54340,60
Virgin Islands,1564,346
